In [1]:
import tensorflow as tf

#1. 모델 만들기

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
#                         (커널(필터) 사이즈)
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

> Conv2D는 다음과 같은 파라미터를 갖는다.
- filters : 32의 출력 결과를 얻는다.
- kernel_size = (3, 3)의 kernel을 사용하여 filetering하게 된다.
- activation = 'relu'라는 활성화 함수를 이용한다.
- input_shape = (28, 28, 1)의 입력 데이터를 이용한다.

> MaxPooling2D는 지정된 크기에서 가장 큰 값을 추출하는 코드로 현재는 2 * 2공간 중 가장 큰 값을 추출하여 새로운 출력 데이터를 만들게 된다.

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
Total params: 37,568
Trainable params: 37,568
Non-trainable params: 0
_________________________________________________________________


> Conv2D의 output은 (input_shape - kernel_size + 1)의 결과를 얻는다.

> 그래서 28*28의 이미지가 Conv2D를 거쳐 26*26이 된다.

>  max_pooling2D는 input_shape / fileter_size로 이전 Conv2D의 output의 결과를 input_shape으로 받아들여 2,2로 나눈 결과가 됨으로, 26/2 = 13이라는 결과를 얻게 된다. (13*13)

In [4]:
model.add(layers.Flatten())

> 처리된 마지막 layer를 1차원으로 변경한다. 위의 경우 13, 13, 32의 output을 갖게 됨으로, 1차원으로 변경하면 13 * 13* 32 = 5408로 변경되는 것이다.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
Total params: 37,568
Trainable params: 37,568
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                102464    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

#2. 데이터 셋 얻기

##2.1 데이터 셋 얻기

In [8]:
from keras.datasets import mnist
(train_data, train_layers), (test_data, test_layers) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [9]:
train_data.shape

(60000, 28, 28)

## 2.2 데이터 변환

In [10]:
X_train = train_data.reshape(60000, 28, 28, 1)
X_train = X_train.astype('float32')/255

X_test = test_data.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')/255

In [11]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_layers)
y_test = to_categorical(test_layers)

In [13]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

#3. 모델 학습 및 평가

##3.1 모델 컴파일

In [ ]:
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
with tf.device('/device:GPU:0'):
  history = model.fit(
      X_train, y_train,
      batch_size=128, epochs=5,
      validation_data=(X_test, y_test)
  ).

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.1975 - accuracy: 0.9395 - val_loss: 0.0636 - val_accuracy: 0.9789
Epoch 2/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0537 - accuracy: 0.9834 - val_loss: 0.0419 - val_accuracy: 0.9866
Epoch 3/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0363 - accuracy: 0.9888 - val_loss: 0.0701 - val_accuracy: 0.9765
Epoch 4/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0279 - accuracy: 0.9914 - val_loss: 0.0346 - val_accuracy: 0.9900
Epoch 5/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0224 - accuracy: 0.9931 - val_loss: 0.0286 - val_accuracy: 0.9908


In [ ]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0286 - accuracy: 0.9908
